In [ ]:
from IPython.display import display
from ipyleaflet import Map, Marker, MarkerCluster, LayerGroup, LayersControl, basemaps, basemap_to_tiles
from ipywidgets import HTML, Layout
from pathlib import Path
import numpy as np
import pandas as pd
import glob
import os
import re

# Resources
zip_to_coordinates = pd.read_csv('./resources/zip_to_coordinates.csv', dtype={'Zip/Postal Code': 'str'})

# Converts Zip+4 to just Zip
def reduceZipCode(zip):
    matches = re.search("^(\d{3,5})", str(zip))
    if matches:
        return matches.group(1)
    return None

def createExport(file, report_title):
    contacts = pd.read_csv(file, dtype={'Zip/Postal Code': 'str', 'Email': 'str'})
    contacts['Zip/Postal Code'] = contacts['Zip/Postal Code'].apply(reduceZipCode)
    contacts = pd.merge(contacts, zip_to_coordinates, on='Zip/Postal Code')
    contacts = contacts.replace(np.nan, '', regex=True)
    return contacts

# Concatenate CSV files in ./input 
report_files = glob.glob(os.path.join("./input", "*.csv"))
for file in report_files:
    report_title = Path(file).stem
    contacts = createExport(file, report_title)

    m = Map(
        basemap=basemaps.CartoDB.Positron,
        center=(39, -95),
        zoom=4,
    )

    markers = []
    for i, row in contacts.iterrows():
        markers.append(Marker(
            location=(row.LAT, row.LNG),
            popup=HTML(f"<div>{row['First Name']} {row['Last Name']}<br />{str(row['Email'])}</div><div><small>{row['Chapter']} {row['Status']}</small></div><div><small>{row['City']}, {row['State']}</small></div>"),
            draggable=False
        ))

    marker_cluster = MarkerCluster(
        name="Clustered Markers",
        markers=markers,
        max_cluster_radius=1
    )

    m.add_layer(marker_cluster)

    # Save to file
    m.layout.height = '80vh'
    m.save(f"./output/{report_title}.html", title=report_title)

    # Show output on completion
    m.layout.height = '300px'
    display(HTML(f"<h1>{report_title}"))
    display(contacts.head(5))
    display(m)
    display(HTML("<hr />"))
